# EFL Data Colloector
need to run EFL.py first

In [910]:
%run 'EFL.py'

## Training Engine
Machines are trained here.

In [1521]:
%run 'EFL.py'
machine = Machine(
    IsingModel(SSLattice(width=32,depth=100,pattern=[-1,-1,1])),
    FreeFermion(size=64,mass=0.,c=4.0),method='weighted')
try:
    machine.load()
except:
    print('File not found! Train new machine.')

INFO:tensorflow:Restoring parameters from ./machine/Ising(SSLatt(32,100,UUI))FF(64,0.00,4.0)weighted


In [1526]:
for i in range(5):
    machine.train(16, learning_rate=0.002, beta1=0.9, beta2=0.9)
    print(machine.session.run((machine.model.MSE, machine.model.J), machine.feed))

(0.00023779711850239443, array([ 0.410,  0.249,  0.249,  0.250,  0.250]))
(0.00057236716773512218, array([ 0.407,  0.252,  0.252,  0.253,  0.254]))
(0.00099527551572211811, array([ 0.406,  0.257,  0.257,  0.257,  0.258]))
(0.00094115121328664593, array([ 0.402,  0.257,  0.257,  0.257,  0.258]))
(0.00046006780626586813, array([ 0.400,  0.254,  0.254,  0.255,  0.255]))


I/O: save machine

In [1327]:
machine.save()

INFO:tensorflow:Saving parameters to ./machine/Ising(SSLatt(32,6,UI))FF(64,0.00,4.0)multiple


----

In [1057]:
#machine.session.run(machine.model.J.assign(np.array([ 0.157,  0.064,  0.000,  0.000,  0.000])))

In [1385]:
print(machine.model.h)
print(machine.session.run(machine.model.J).tolist())

2.77258872224
[0.38646672076585065, 0.2087161408620652, 0.20843718493409727, 0.20716343536763865, 0.20562131964588898]


## J Sequence
Given trained machine, run additional trainings to get the time sequence of J.
### Collect Jseq
Data format: `Jseq = {machine: [[J1,J2,...], ...], ...}`

In [1357]:
try:
    Jseq = load('Jseq')
except FileNotFoundError:
    Jseq = {}
try:
    Js = Jseq[machine.info]
except KeyError:
    Js = []
print('len: ', len(Js))

len:  20


In [1373]:
for i in range(20):
    machine.train(16, learning_rate=0.002, check=16)
    print(machine.session.run((machine.model.MSE, machine.model.J), machine.feed))
    Js.append(list(machine.session.run(machine.model.J)))
print('len: ', len(Js))
print('avg: ', np.mean(Js,axis=0))
print('std: ', np.std(Js,axis=0))

(0.00033694051017811767, array([ 0.381,  0.211,  0.211,  0.211,  0.211]))
(0.00020964445685019782, array([ 0.379,  0.207,  0.207,  0.207,  0.207]))
(0.00039868853881976481, array([ 0.380,  0.210,  0.210,  0.210,  0.210]))
(0.00058209832862644059, array([ 0.380,  0.210,  0.210,  0.210,  0.210]))
(0.00018487138009892925, array([ 0.380,  0.210,  0.210,  0.210,  0.210]))
(0.00036805942655325827, array([ 0.380,  0.208,  0.208,  0.208,  0.209]))
(0.00037006609461553481, array([ 0.381,  0.208,  0.208,  0.208,  0.208]))
(0.0003883804133829891, array([ 0.380,  0.207,  0.207,  0.208,  0.208]))
(0.00023534368346915074, array([ 0.379,  0.207,  0.207,  0.207,  0.207]))
(0.00055085141682146199, array([ 0.381,  0.209,  0.209,  0.209,  0.209]))
(0.00042298703230318036, array([ 0.379,  0.207,  0.207,  0.207,  0.207]))
(0.00074563947084712798, array([ 0.380,  0.208,  0.208,  0.208,  0.208]))
(0.00058330246595166425, array([ 0.382,  0.208,  0.208,  0.208,  0.209]))
(0.00059395344159677145, array([ 0.382,

In [1374]:
Jseq[machine.info] = Js
save('Jseq', Jseq)

----

In [1368]:
Js=[]

In [907]:
#sorted([(mach, np.mean(Js,axis=0)) for mach, Js in Jseq.items()]);

In [1331]:
sorted(list(Jseq.keys()));

### Export J data 

In [1377]:
clst = [0.5,1.,2.,4.,8.,16.]
Jseq = load('Jseq')
data = []
for c in clst:
    _machine = Machine(
        IsingModel(SSLattice(width=32,depth=100,pattern=[-1,-1,1])),
        FreeFermion(size=64,mass=1e-8,c=c),method='random')
    _Js = Jseq[_machine.info]
    Jmean = np.mean(_Js,axis=0)
    Jstd = np.std(_Js,axis=0)
    _machine.build()
    _machine.session.run(_machine.model.J.assign(Jmean))
    MSE = _machine.session.run(_machine.model.MSE, _machine.data_server.fetch('random', batch=64))
    data.append([c, np.asscalar(np.sqrt(MSE)), Jmean.tolist(), Jstd.tolist()])
export('ccurv_r', data)

In [1228]:
masslst = [0.,0.01,0.02,0.03,0.04,0.05,0.07,0.1,0.12,0.15,0.17,0.2,0.25,0.3,0.35,0.4,0.5]
Jseq = load('Jseq')
data = []
for mass in masslst:
    info = Machine(
        IsingModel(SSLattice(width=32,depth=100,pattern=[-1,-1,1])),
        FreeFermion(size=64,mass=mass,c=4.0),method='weighted').info
    Jstmp = Jseq[info]
    data.append([mass, np.mean(Jstmp,axis=0).tolist()])
export('mmap_c4', data)

In [1232]:
masslst = [0.,0.05,0.1,0.2,0.5]
Jseq = load('Jseq')
data = []
for mass in masslst:
    info = Machine(
        IsingModel(SSLattice(width=32,depth=100,pattern=[-1,-1,1])),
        FreeFermion(size=64,mass=mass,c=4.0),method='weighted').info
    Jstmp = Jseq[info]
    data.append([mass, np.mean(Jstmp,axis=0).tolist(), np.std(Jstmp,axis=0).tolist()])
export('mcurv_c4', data)

## Entanglement Feature
Given trained machine, collect entropy predictions
### Collect EFcon
Entanglement feature over consecutive regions.
Data format: `EFcon = {machine: {'Ssys': [S1,...], 'Smdls': {[[S1,...], ...]}}, ...}`

In [1323]:
try:
    EFcon = load('EFcon')
except FileNotFoundError:
    EFcon = {}
try:
    EF = EFcon[machine.info]
    Ssys = EF['Ssys']
    Smdls = EF['Smdls']
except KeyError:
    Ssys = (machine.data_server.fetch('consecutive')[machine.model.Ssys]).tolist()
    Smdls = []
    EF = {'Ssys': Ssys, 'Smdls': Smdls}
print('len: ', len(Smdls))

len:  0


In [1324]:
for i in range(5):
    machine.train(1, learning_rate=0.002, check=16)
    print(machine.session.run((machine.model.MSE, machine.model.J), machine.feed))
    Smdl = machine.session.run(machine.model.Smdl, 
                            machine.data_server.fetch('consecutive')).tolist()
    Smdls.append(Smdl)
print('len: ', len(Smdls))
print('error: ', np.mean([np.sqrt(np.mean(np.square(np.array(Smdl)/np.array(Ssys)-1))) for Smdl in Smdls]))

(0.00052611360252749993, array([ 0.337,  0.319,  0.319,  0.320,  0.320]))
(0.0010102109143726723, array([ 0.336,  0.319,  0.319,  0.319,  0.320]))
(0.0014639201730284642, array([ 0.336,  0.318,  0.318,  0.319,  0.319]))
(0.0008748755622376732, array([ 0.335,  0.318,  0.318,  0.318,  0.318]))
(0.0012509499391978288, array([ 0.335,  0.317,  0.317,  0.318,  0.318]))
len:  5
error:  0.0378897835945


In [1325]:
EFcon[machine.info] = {'Ssys': Ssys, 'Smdls': Smdls}
save('EFcon', EFcon)

### Collect EFmul
Entanglement feature over multiregions.
Data format: `EFmul = {machine: {n: [[Ssys, Smdl], ...], ...},...}`

In [1322]:
try:
    EFmul = load('EFmul')
except FileNotFoundError:
    EFmul = {}
try:
    ndict = EFmul[machine.info]
except KeyError:
    ndict = {n:[] for n in range(1, machine.data_server.partitions//2+1)}
print('len: ', len(ndict[1]))

len:  35


In [1319]:
for i in range(5):
    machine.train(16, check=16)
    print(machine.session.run((machine.model.MSE, machine.model.J), machine.feed))
    for n, pairs in ndict.items():
        Ssysn, Smdln = machine.session.run([
                        machine.model.Ssys, machine.model.Smdl], 
                        machine.data_server.fetch(('multiple',n)))
        Ssysn = Ssysn.tolist()
        Smdln = Smdln.tolist()
        pairs += [[Ssys, Smdl] for Ssys, Smdl in zip(Ssysn, Smdln)]
print('len: ', len(ndict[1]))
print({n:float('{0:.3f}'.format(np.sqrt(np.mean([(a/b-1.)**2 for [a, b] in pairs])))) for n, pairs in ndict.items()})

(0.0017530258897438119, array([ 0.337,  0.320,  0.320,  0.320,  0.321]))
(0.0012464493350100344, array([ 0.337,  0.320,  0.320,  0.320,  0.321]))
(0.0010929459295898377, array([ 0.338,  0.320,  0.320,  0.321,  0.321]))
(0.0018914511421001306, array([ 0.338,  0.320,  0.320,  0.321,  0.321]))
(0.0021255521544174837, array([ 0.337,  0.320,  0.320,  0.320,  0.320]))
len:  35
{1: 0.037, 2: 0.044, 3: 0.05, 4: 0.052, 5: 0.048, 6: 0.051, 7: 0.046, 8: 0.042, 9: 0.043, 10: 0.045, 11: 0.046, 12: 0.045, 13: 0.047, 14: 0.049, 15: 0.05, 16: 0.052}


In [1320]:
EFmul[machine.info] = ndict
save('EFmul', EFmul)

In [1488]:
#for mach, ndict in EFmul.items():
#    print(mach,{n:float('{0:.3f}'.format(np.sqrt(np.mean([(a/b-1.)**2 for [a, b] in pairs])))) for n, pairs in ndict.items()})

### Export Entanglement Feature

In [1256]:
EFcon = load('EFcon')
info = Machine(
        IsingModel(SSLattice(width=32,depth=100,pattern=[-1,-1,1])),
        FreeFermion(size=64,mass=0.,c=4.0),method='multiple').info
export('EFcon_c4', EFcon[info])

In [1257]:
EFmul = load('EFmul')
info = Machine(
        IsingModel(SSLattice(width=32,depth=100,pattern=[-1,-1,1])),
        FreeFermion(size=64,mass=0.,c=4.0),method='multiple').info
export('EFmul_c4', EFmul[info])

In [1326]:
export('EFcon_c4_UI', EFcon[machine.info])

## Mutual Information

A, B are single block regions, they separate by distance $d$ (blocks). The short rangle mutual information still fits, but very quickly the accuracy lost as the separation is grater than the region size.

In [1527]:
class MIcollector():
    def __init__(self, machine):
        self.machine = machine
        self.partition = machine.data_server.partitions
    def Ssys(self, region):
        sites = self.machine.data_server.sites(region)
        S = self.machine.system.S(sites)
        return S
    def Smdl(self, region):
        confs = np.array([region.config()])
        feed = {self.machine.model.confs: confs}
        [S] = self.machine.session.run(self.machine.model.Smdl, feed).tolist()
        return S
    def Ilst(self, S):
        res = []
        A = Region({0}, self.partition)
        SA = S(A)
        for d in range(1, self.partition//2):
            AB = Region({0,d}, self.partition)
            IAB = 2*SA - S(AB)
            res.append(IAB)
        return res

In [1528]:
mic = MIcollector(machine)
list(zip(mic.Ilst(mic.Ssys), mic.Ilst(mic.Smdl)))

[(2.057311443226857, 2.3939432955080235),
 (0.33647191417722766, 0.24722113107637256),
 (0.14034130133619982, 0.69693783012886),
 (0.07854071988457623, 0.010964931360177843),
 (0.05110174148634794, 0.6640045833051147),
 (0.03653593162731994, 0.0016195094607098781),
 (0.02790473026868945, 0.6880268611519966),
 (0.022401011788256042, 0.00019241183369445025),
 (0.01871059202127956, 0.6627417363334587),
 (0.01615189730513844, -0.002194407640075724),
 (0.014344089344984212, 0.6860237879531326),
 (0.013062597084608996, -0.0021897337741449974),
 (0.012170281988280962, 0.662744097523472),
 (0.011582366519046872, 0.00012598098820149062),
 (0.011247772007068058, 0.6879633783148336),
 (0.011139116829002482, 0.00010640634602054888)]

Let us try the mutual information of neighboring regions.

In [1562]:
class MIcollector():
    def __init__(self, machine, shift = 1):
        self.machine = machine
        self.shift = shift
        self.partition = machine.data_server.partitions
    def Ssys(self, region):
        sites = self.machine.data_server.sites(region)
        S = self.machine.system.S(sites)
        return S
    def Smdl(self, region):
        confs = np.array([region.config()])
        feed = {self.machine.model.confs: confs}
        [S] = self.machine.session.run(self.machine.model.Smdl, feed).tolist()
        return S
    def Ilst(self, S):
        res = []
        for LA in range(1, self.partition//(self.shift+1)):
            A = Region(range(LA), self.partition)
            SA = S(A)
            #AB = Region({range(LA)}, self.partition)
            AB = Region({x for x in range(LA)}^{x+self.shift*LA for x in range(LA)}, self.partition)
            IAB = 2*SA - S(AB)
            res.append(IAB)
        return res

In [1564]:
mic = MIcollector(machine,2)
list(zip(mic.Ilst(mic.Ssys), mic.Ilst(mic.Smdl)))

[(0.33647191417722766, 0.24821014626670035),
 (0.33577169617316205, 0.10917166428023961),
 (0.3459704449498462, 0.3300445061934738),
 (0.36945531926099484, 0.11767144474021052),
 (0.4084436991771252, 0.17150139682968302),
 (0.4688213361532352, 0.2500264392829479),
 (0.5628912765582985, 0.3573247622033193),
 (0.7177686195927588, 0.3708387872150922),
 (1.0062721483062465, 0.7140448212021511)]

### Collect MIadj
Mutual information over adjacent regions.
Data format: `MIadj = {machine: {'Isys': [I1,...], 'Imdls': {[[I1,...], ...]}}, ...}`

In [1549]:
mic = MIcollector(machine)
try:
    MIadj = load('MIadj')
except FileNotFoundError:
    MIadj = {}
try:
    MI = MIadj[machine.info]
    Isys = MI['Isys']
    Imdls = MI['Imdls']
except KeyError:
    Isys = mic.Ilst(mic.Ssys)
    Imdls = []
    MI = {'Isys': Isys, 'Imdls': Imdls}
print('len: ', len(Imdls))

len:  0


In [1553]:
for i in range(5):
    machine.train(16, learning_rate=0.002)
    print(machine.session.run((machine.model.MSE, machine.model.J), machine.feed))
    Imdl = mic.Ilst(mic.Smdl)
    Imdls.append(Imdl)
print('len: ', len(Imdls))
print('error: ', np.mean([np.sqrt(np.mean(np.square(np.array(Imdl)/np.array(Isys)-1))) for Imdl in Imdls]))

(0.00056119298030312401, array([ 0.404,  0.254,  0.254,  0.254,  0.255]))
(0.0011791493723503652, array([ 0.404,  0.252,  0.252,  0.253,  0.253]))
(0.00040430123766450361, array([ 0.404,  0.253,  0.253,  0.253,  0.254]))
(0.0015620737802473576, array([ 0.404,  0.258,  0.258,  0.258,  0.259]))
(0.0018916727227648138, array([ 0.402,  0.254,  0.254,  0.254,  0.255]))
len:  20
error:  0.0551127607659


In [1554]:
MIadj[machine.info] = {'Isys': Isys, 'Imdls': Imdls}
save('MIadj', MIadj)

### Collect MIadj2
Mutual information over adjacent regions.
Data format: `MIadj = {machine: {'Isys': [I1,...], 'Imdls': {[[I1,...], ...]}}, ...}`

In [1565]:
mic = MIcollector(machine,2)
try:
    MIadj = load('MIadj2')
except FileNotFoundError:
    MIadj = {}
try:
    MI = MIadj[machine.info]
    Isys = MI['Isys']
    Imdls = MI['Imdls']
except KeyError:
    Isys = mic.Ilst(mic.Ssys)
    Imdls = []
    MI = {'Isys': Isys, 'Imdls': Imdls}
print('len: ', len(Imdls))

len:  0


In [1569]:
for i in range(5):
    machine.train(16, learning_rate=0.002)
    print(machine.session.run((machine.model.MSE, machine.model.J), machine.feed))
    Imdl = mic.Ilst(mic.Smdl)
    Imdls.append(Imdl)
print('len: ', len(Imdls))
print('error: ', np.mean([np.sqrt(np.mean(np.square(np.array(Imdl)/np.array(Isys)-1))) for Imdl in Imdls]))

(0.0013166201966903796, array([ 0.407,  0.250,  0.250,  0.251,  0.251]))
(0.0011122525847786853, array([ 0.410,  0.252,  0.252,  0.252,  0.253]))
(0.00047054566896786765, array([ 0.410,  0.253,  0.253,  0.253,  0.254]))
(0.001778112361748434, array([ 0.408,  0.251,  0.251,  0.252,  0.253]))
(0.00072705443348058138, array([ 0.405,  0.255,  0.255,  0.256,  0.256]))
len:  20
error:  0.471544330592


In [1571]:
MIadj[machine.info] = {'Isys': Isys, 'Imdls': Imdls}
save('MIadj2', MIadj)

### Export Mutual Information

In [1572]:
MIadj = load('MIadj')
info = Machine(
        IsingModel(SSLattice(width=32,depth=100,pattern=[-1,-1,1])),
        FreeFermion(size=64,mass=0.,c=4.0),method='weighted').info
export('MIadj_c4', MIadj[info])

In [ ]:
MIadj = load('MIadj2')
info = Machine(
        IsingModel(SSLattice(width=32,depth=100,pattern=[-1,-1,1])),
        FreeFermion(size=64,mass=0.,c=4.0),method='weighted').info
export('MIadj2_c4', MIadj[info])